# TSA Chapter 6: Case Study - Rolling Forecast VAR vs AR

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch6/TSA_ch6_case_rolling_forecast/TSA_ch6_case_rolling_forecast.ipynb)

Rolling one-step-ahead forecasts comparing VAR and AR(1) models.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download interest rate data for cointegration case study
try:
    gs10 = web.DataReader('GS10', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    gs2 = web.DataReader('GS2', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    rates = pd.DataFrame({'10Y': gs10.iloc[:,0], '2Y': gs2.iloc[:,0]}).dropna()
except:
    np.random.seed(42); n = 408; dates = pd.date_range('1990-01-01', periods=n, freq='MS')
    rw = np.cumsum(np.random.normal(0, 0.1, n)) + 5
    rates = pd.DataFrame({'10Y': rw + np.random.normal(0, 0.2, n), '2Y': rw - 0.5 + np.random.normal(0, 0.15, n)}, index=dates)
print(f'Rates data: {len(rates)} obs')

diff_rates = rates.diff().dropna()
n_test = 120
var_fcast = {c: [] for c in diff_rates.columns}
ar_fcast = {c: [] for c in diff_rates.columns}

for i in range(n_test):
    train = diff_rates.iloc[:len(diff_rates)-n_test+i]
    # VAR forecast
    try:
        res = VAR(train).fit(2)
        fcast = res.forecast(train.values[-2:], steps=1)
        for j, c in enumerate(diff_rates.columns): var_fcast[c].append(fcast[0, j])
    except:
        for c in diff_rates.columns: var_fcast[c].append(0)
    # AR(1) forecast
    for j, c in enumerate(diff_rates.columns):
        try:
            ar_res = AutoReg(train[c], lags=1).fit()
            ar_fcast[c].append(ar_res.forecast(1).values[0])
        except:
            ar_fcast[c].append(0)

test_idx = diff_rates.index[-n_test:]
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, col, c in zip(axes, diff_rates.columns, [BLUE, RED]):
    actual = diff_rates[col].values[-n_test:]
    var_rmse = np.sqrt(np.mean((actual - np.array(var_fcast[col]))**2))
    ar_rmse = np.sqrt(np.mean((actual - np.array(ar_fcast[col]))**2))
    ax.plot(range(len(actual)), actual, color=GRAY, lw=0.5, alpha=0.6, label='Actual')
    ax.plot(range(len(actual)), var_fcast[col], color=BLUE, lw=0.8, ls='--', label='VAR Forecast')
    ax.plot(range(len(actual)), ar_fcast[col], color=RED, lw=0.8, ls=':', label='AR(1) Forecast')
    ax.set_title(f'$\\Delta${col}\nVAR RMSE={var_rmse:.6f}, AR RMSE={ar_rmse:.6f}', fontweight='bold', fontsize=10)
    ax.set_xlabel('Observation')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

plt.tight_layout(); save_chart(fig, 'ch6_case_rolling_forecast'); plt.show()